### 💻 **ALL IMPORT**

In [1]:
import numpy as np
import pandas as pd
import requests
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import contextily as ctx
from matplotlib.colors import to_rgba

import folium
from folium.plugins import HeatMap
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time
from PIL import Image
import seaborn as sns


### 💻 **REQUETTE API**

In [2]:
all_records = pd.read_csv('les-arbres.csv', sep=';')

# Exemple d'affichage des premiers enregistrements
all_records.head()

,IDBASE,TYPE EMPLACEMENT,DOMANIALITE,ARRONDISSEMENT,COMPLEMENT ADRESSE,NUMERO,LIEU / ADRESSE,IDEMPLACEMENT,LIBELLE FRANCAIS,GENRE,ESPECE,VARIETE OUCULTIVAR,CIRCONFERENCE (cm),HAUTEUR (m),STADE DE DEVELOPPEMENT,REMARQUABLE,geo_point_2d
0,2036715,Arbre,Jardin,PARIS 10E ARRDT,NaN,NaN,SQUARE ALBAN SATRAGNE / 11 SQUARE ALBAN SATRAGNE,000202003,Cerisier à fleurs,Prunus,serrulata,''Fugenzo'',20,3,Jeune (arbre),NON,"48.875678916820775, 2.354826317511796"
1,213055,Arbre,Alignement,PARIS 12E ARRDT,NaN,NaN,BOULEVARD DE LA GUYANE,000301065,Noisetier de Byzance,Corylus,colurna,NaN,55,6,Jeune (arbre),NON,"48.84364564591775, 2.414494970152308"
2,226874,Arbre,Alignement,PARIS 14E ARRDT,205,NaN,AVENUE DU MAINE,003101010,Platane,Platanus,x hispanica,NaN,105,15,Jeune (arbre)Adulte,NON,"48.82903929799474, 2.3265411925175825"
3,101372,Arbre,Jardin,PARIS 12E ARRDT,LES PARTERRES - La maison du jardinage,NaN,PARC DE BERCY - JARDIN YITZHAK RABIN / 16 RUE ...,B00404002,Cognassier,Cydonia,oblonga,NaN,90,5,Adulte,NON,"48.835747448399054, 2.382080336441387"
4,106808,Arbre,Jardin,PARIS 7E ARRDT,Canton 12,NaN,JARDIN DU CHAMP DE MARS ET PELOUSES DE L ECOLE...,A01202004,NaN,Tilia,n. sp.,NaN,125,13,Adulte,NON,"48.85526310210728, 2.301680194864072"


### 🔎 **TYPE VALEUR**

Variable Quantitatives
- Discrètes => Idbase / numero
- Continue => circonference / hauteur / geo_point_2d 

Variable Quaalitatives
- Nominale => type emplacement / domanialite / complement adresse / lieu adresse / idemplacement / libelle français / genre / espece / variete oucultivar
 / remarquables / arrondissement
 - Ordinale => stade de développement 

### 🔎 **VALEUR NULL**

In [3]:
# Calculer le nombre total de valeurs nulles par colonne
null_counts = all_records.isnull().sum()

# Obtenir le nombre total de lignes
nombre_total_de_lignes = all_records.shape[0]

# Aperçu
print("Total element:", nombre_total_de_lignes)
print(null_counts)

Total element: 210296
IDBASE                         0
TYPE EMPLACEMENT               0
DOMANIALITE                    0
ARRONDISSEMENT                 0
COMPLEMENT ADRESSE        149028
NUMERO                    210296
LIEU / ADRESSE                 0
IDEMPLACEMENT                  0
LIBELLE FRANCAIS           12897
GENRE                          0
ESPECE                      2772
VARIETE OUCULTIVAR        170013
CIRCONFERENCE (cm)             0
HAUTEUR (m)                    0
STADE DE DEVELOPPEMENT     47640
REMARQUABLE                21038
geo_point_2d                   0
dtype: int64


In [ ]:
# Filtrer les lignes avec des valeurs nulles (sauf pour la colonne 'NUMERO') et éliminer les doublons
columns_to_check = all_records.columns.difference(['NUMERO', 'COMPLEMENT ADRESSE', 'REMARQUABLE', 'VARIETE OUCULTIVAR'])  # Exclure la colonne 'NUMERO'
elements_avec_null = all_records[all_records[columns_to_check].isnull().any(axis=1)].drop_duplicates()


# Vérifier et filtrer les valeurs NaN dans 'geo_point_2d'
elem_null = elements_avec_null.dropna(subset=['geo_point_2d'])

# Extraire les coordonnées GPS et convertir en float
elem_null[['lat', 'lon']] = elem_null['geo_point_2d'].str.split(', ', expand=True).astype(float)

# Filtrer les lignes avec des valeurs NaN après la séparation des coordonnées
elem_null = elem_null.dropna(subset=['lat', 'lon'])

# Créer des points géométriques
elem_null['geometry'] = elem_null.apply(lambda row: Point(row['lon'], row['lat']), axis=1)

# Créer un GeoDataFrame
gdf = gpd.GeoDataFrame(elem_null, geometry='geometry', crs="EPSG:4326")

# Projetter en Mercator pour l'utilisation avec contextily
gdf = gdf.to_crs(epsg=3857)

# Créer une figure et un axe
fig, ax = plt.subplots(1, 1, figsize=(12, 8))

# Créer la heatmap
gdf.plot(ax=ax, alpha=0.6, edgecolor='k', marker='o', markersize=5)

# Ajouter une couche de fond avec contextily
ctx.add_basemap(ax, crs=gdf.crs.to_string(), source=ctx.providers.OpenStreetMap.Mapnik)

# Ajouter une légende
plt.title(f'Localisation des arbres contenant une valeur null.\nTotal: {len(elements_avec_null)}', ha='center')
plt.xlabel('Longitude')
plt.ylabel('Latitude')

# Sauvegarder la carte en tant qu'image
plt.savefig("outliers_heatmap.png", dpi=300, bbox_inches='tight')

# Afficher la carte
plt.show()

In [4]:
# Calculer le nombre total de valeurs double par colonne
null_counts = all_records.loc[all_records['IDBASE'].duplicated(keep=False),:]

# Aperçu
print(null_counts)

         IDBASE TYPE EMPLACEMENT DOMANIALITE  ARRONDISSEMENT  \
2511    2009124            Arbre  Alignement  PARIS 4E ARRDT   
78967    227160            Arbre  Alignement  PARIS 6E ARRDT   
172885   227160            Arbre  Alignement  PARIS 6E ARRDT   
204160  2009124            Arbre  Alignement  PARIS 4E ARRDT   

       COMPLEMENT ADRESSE  NUMERO                          LIEU / ADRESSE  \
2511                  NaN     NaN                            QUAI D ANJOU   
78967                 NaN     NaN  PORT DES SAINTS PERES / QUAI MALAQUAIS   
172885                NaN     NaN  PORT DES SAINTS PERES / QUAI MALAQUAIS   
204160                NaN     NaN                            QUAI D ANJOU   

       IDEMPLACEMENT LIBELLE FRANCAIS    GENRE  ESPECE VARIETE OUCULTIVAR  \
2511       000202007              NaN  Populus     NaN                NaN   
78967      000201008         Peuplier  Populus    alba          ''Raket''   
172885     000201008         Peuplier  Populus    alba        

In [5]:
# SUPPRESSION DONNÉE CAR VALEURS DIFFERENTE


# Identifier les valeurs dupliquées dans 'IDBASE'
duplicates = all_records['IDBASE'].duplicated(keep=False)
duplicated_values = all_records.loc[duplicates, 'IDBASE'].unique()

# Filtrer le DataFrame pour supprimer toutes les lignes contenant des valeurs dupliquées
df_no_duplicates = all_records[~all_records['IDBASE'].isin(duplicated_values)]

# Vérification doublon supprimer
print(df_no_duplicates[df_no_duplicates['IDBASE'] == 2009124])
print(df_no_duplicates[df_no_duplicates['IDBASE'] == 227160])

Empty DataFrame
Columns: [IDBASE, TYPE EMPLACEMENT, DOMANIALITE, ARRONDISSEMENT, COMPLEMENT ADRESSE, NUMERO, LIEU / ADRESSE, IDEMPLACEMENT, LIBELLE FRANCAIS, GENRE, ESPECE, VARIETE OUCULTIVAR, CIRCONFERENCE (cm), HAUTEUR (m), STADE DE DEVELOPPEMENT, REMARQUABLE, geo_point_2d]
Index: []
Empty DataFrame
Columns: [IDBASE, TYPE EMPLACEMENT, DOMANIALITE, ARRONDISSEMENT, COMPLEMENT ADRESSE, NUMERO, LIEU / ADRESSE, IDEMPLACEMENT, LIBELLE FRANCAIS, GENRE, ESPECE, VARIETE OUCULTIVAR, CIRCONFERENCE (cm), HAUTEUR (m), STADE DE DEVELOPPEMENT, REMARQUABLE, geo_point_2d]
Index: []


### 🔎 **VALEUR NUMBER**

In [6]:
# Vérifier que les valeurs dans les colonnes 'CIRCONFERENCE' et 'HAUTEUR' sont des nombres
def is_numeric(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

# Utiliser loc pour éviter SettingWithCopyWarning
df_no_duplicates.loc[:, 'is_circonference_numeric'] = df_no_duplicates['CIRCONFERENCE (cm)'].apply(is_numeric)
df_no_duplicates.loc[:, 'is_hauteur_numeric'] = df_no_duplicates['HAUTEUR (m)'].apply(is_numeric)

# Filtrer les lignes où les valeurs ne sont pas numériques
non_numeric_circonference = df_no_duplicates.loc[~df_no_duplicates['is_circonference_numeric'], ['IDBASE', 'CIRCONFERENCE (cm)']]
non_numeric_hauteur = df_no_duplicates.loc[~df_no_duplicates['is_hauteur_numeric'], ['IDBASE', 'HAUTEUR (m)']]

# Afficher les résultats
print("Lignes avec valeurs non numériques dans 'CIRCONFERENCE (cm)':")
print(non_numeric_circonference)

print("\nLignes avec valeurs non numériques dans 'HAUTEUR (m)':")
print(non_numeric_hauteur)

# Supprimer les colonnes temporaires utilisées pour la vérification
df_no_duplicates = df_no_duplicates.drop(columns=['is_circonference_numeric', 'is_hauteur_numeric'])

Lignes avec valeurs non numériques dans 'CIRCONFERENCE (cm)':
Empty DataFrame
Columns: [IDBASE, CIRCONFERENCE (cm)]
Index: []

Lignes avec valeurs non numériques dans 'HAUTEUR (m)':
Empty DataFrame
Columns: [IDBASE, HAUTEUR (m)]
Index: []


C:\Users\Nerion\AppData\Local\Temp\ipykernel_34332\851948130.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_duplicates.loc[:, 'is_circonference_numeric'] = df_no_duplicates['CIRCONFERENCE (cm)'].apply(is_numeric)
C:\Users\Nerion\AppData\Local\Temp\ipykernel_34332\851948130.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_duplicates.loc[:, 'is_hauteur_numeric'] = df_no_duplicates['HAUTEUR (m)'].apply(is_numeric)


In [7]:
# Calculer les statistiques descriptives
stats = df_no_duplicates[['CIRCONFERENCE (cm)', 'HAUTEUR (m)']].describe(percentiles=[.25, .5, .75]).loc[['mean', 'std', 'min', 'max', '25%', '50%', '75%']]

# Afficher les résultats
print(stats)

      CIRCONFERENCE (cm)  HAUTEUR (m)
mean           80.804372     8.837288
std            63.377643     5.992876
min             0.000000     0.000000
max          2280.000000   225.000000
25%            30.000000     5.000000
50%            70.000000     8.000000
75%           115.000000    12.000000


In [140]:
# Calculer les quartiles et l'IQR pour chaque colonne
Q1 = df_no_duplicates[['CIRCONFERENCE (cm)', 'HAUTEUR (m)']].quantile(0.25)
Q3 = df_no_duplicates[['CIRCONFERENCE (cm)', 'HAUTEUR (m)']].quantile(0.75)
IQR = Q3 - Q1

# Définir les limites pour les outliers
lower_bound = 0.3 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filtrer les données pour enlever les outliers
filtered_df = df_no_duplicates[~((df_no_duplicates[['CIRCONFERENCE (cm)', 'HAUTEUR (m)']] < lower_bound) | (df_no_duplicates[['CIRCONFERENCE (cm)', 'HAUTEUR (m)']] > upper_bound)).any(axis=1)]

# Filtrer les données pour obtenir les outliers
outliers_df = df_no_duplicates[((df_no_duplicates[['CIRCONFERENCE (cm)', 'HAUTEUR (m)']] < lower_bound) | (df_no_duplicates[['CIRCONFERENCE (cm)', 'HAUTEUR (m)']] > upper_bound)).any(axis=1)]

# Calculer les statistiques descriptives après filtrage
stats_filtered = filtered_df[['CIRCONFERENCE (cm)', 'HAUTEUR (m)']].describe(percentiles=[.25, .5, .75]).loc[['mean', 'std', 'min', 'max', '25%', '50%', '75%']]

# Afficher les variables
print(lower_bound)
print(upper_bound)
print()

# Afficher les résultats des statistiques descriptives
print(stats_filtered)
print()

# Afficher le nombre de lignes après filtrage
count_filtered = len(filtered_df)
print(f"Nombre total de lignes après filtrage : {count_filtered}")
print()

# Afficher le nombre d'éléments
print(f"Nombre d'Outliers: {len(outliers_df)}")

CIRCONFERENCE (cm)    25.5
HAUTEUR (m)            2.1
dtype: float64
CIRCONFERENCE (cm)    242.5
HAUTEUR (m)            22.5
dtype: float64

      CIRCONFERENCE (cm)  HAUTEUR (m)
mean           95.207966    10.539359
std            47.378570     4.465122
min            26.000000     3.000000
max           242.000000    22.000000
25%            58.000000     7.000000
50%            85.000000    10.000000
75%           125.000000    14.000000

Nombre total de lignes après filtrage : 151578

Nombre d'Outliers: 58714


In [ ]:
# Vérifier et filtrer les valeurs NaN dans 'geo_point_2d'
outliers_df = outliers_df.dropna(subset=['geo_point_2d'])

# Extraire les coordonnées GPS et convertir en float
outliers_df[['lat', 'lon']] = outliers_df['geo_point_2d'].str.split(', ', expand=True).astype(float)

# Filtrer les lignes avec des valeurs NaN après la séparation des coordonnées
outliers_df = outliers_df.dropna(subset=['lat', 'lon'])

# Créer des points géométriques
outliers_df['geometry'] = outliers_df.apply(lambda row: Point(row['lon'], row['lat']), axis=1)

# Créer un GeoDataFrame
gdf = gpd.GeoDataFrame(outliers_df, geometry='geometry', crs="EPSG:4326")

# Projetter en Mercator pour l'utilisation avec contextily
gdf = gdf.to_crs(epsg=3857)

# Créer une figure et un axe
fig, ax = plt.subplots(1, 1, figsize=(12, 8))

# Créer la heatmap
gdf.plot(ax=ax, alpha=0.6, edgecolor='k', marker='o', markersize=5)

# Ajouter une couche de fond avec contextily
ctx.add_basemap(ax, crs=gdf.crs.to_string(), source=ctx.providers.OpenStreetMap.Mapnik)

# Ajouter une légende
plt.title(f'Localisation des arbres à remesurer\nTotal: {len(outliers_df)}', ha='center')
plt.xlabel('Longitude')
plt.ylabel('Latitude')

# Sauvegarder la carte en tant qu'image
plt.savefig("outliers_heatmap.png", dpi=300, bbox_inches='tight')

# Afficher la carte
plt.show()

In [ ]:
# Agrégation des données : Compter le nombre d'IDBASE par arrondissement
arrondissement_counts = outliers_df['ARRONDISSEMENT'].value_counts().reset_index()
arrondissement_counts.columns = ['Arrondissement', 'Nombre d\'IDBASE']

# Créer le graphique
plt.figure(figsize=(10, 10))
barplot = plt.barh(arrondissement_counts['Arrondissement'], arrondissement_counts['Nombre d\'IDBASE'], color='skyblue')

# Ajouter des annotations pour le nombre total par ligne
for index, row in arrondissement_counts.iterrows():
    plt.text(row['Nombre d\'IDBASE'] + 200, index, row['Nombre d\'IDBASE'], color='black', ha="center")

# Ajuster l'espacement entre les graduations des axes
plt.yticks(ticks=range(len(arrondissement_counts)), labels=arrondissement_counts['Arrondissement'], va='center')

# Ajouter les labels et le titre
plt.xlabel('Nombre d\'IDBASE')
plt.ylabel('ARRONDISSEMENT')
plt.title('Quantité d\'arbre à re-mesurer par arrondissement')

# Ajuster l'espacement entre les lignes
plt.gca().yaxis.set_tick_params(pad=0)

# Afficher le graphique
plt.tight_layout()
plt.show()

In [ ]:
# Créer le graphique
g = sns.jointplot(data=filtered_df, x='CIRCONFERENCE (cm)', y='HAUTEUR (m)', hue='STADE DE DEVELOPPEMENT', kind='scatter', palette='Set1', height=8, ratio=2)

# Redimensionner le graphique
g.fig.set_figwidth(12)  # Largeur du graphique
g.fig.set_figheight(8)  # Hauteur du graphique

# Ajouter le titre
plt.suptitle('Hauteur en fonction de la circonférence et le stade de développement des arbres', y=1.02)

plt.show()

In [ ]:
# Palette de couleurs personnalisée
palette_color = {'Jeune (arbre)': 'red', 'Jeune (arbre)Adulte': 'blue', 'Adulte': 'green', 'Mature': 'purple'}


# Créer une figure avec deux sous-graphes
fig, axes = plt.subplots(1, 2, figsize=(14, 7))

# Premier graphique: Hauteur par stade de développement
sns.boxplot(ax=axes[0], x='STADE DE DEVELOPPEMENT', y='HAUTEUR (m)', data=filtered_df, hue='STADE DE DEVELOPPEMENT', palette=palette_color, legend=False)
axes[0].set_title('Hauteur par stade de développement')
axes[0].set_xlabel('STADE DE DEVELOPPEMENT')
axes[0].set_ylabel('HAUTEUR (m)')

# Deuxième graphique: Circonférence par stade de développement
sns.boxplot(ax=axes[1], x='STADE DE DEVELOPPEMENT', y='CIRCONFERENCE (cm)', data=filtered_df, hue='STADE DE DEVELOPPEMENT', palette=palette_color, legend=False)
axes[1].set_title('Circonférence par stade de développement')
axes[1].set_xlabel('STADE DE DEVELOPPEMENT')
axes[1].set_ylabel('CIRCONFERENCE (cm)')

# Ajuster l'espacement entre les sous-graphes
plt.tight_layout()

# Afficher les graphiques
plt.show()

In [ ]:
# Compter les occurrences de chaque genre
genre_counts = filtered_df['GENRE'].value_counts()

# Sélectionner les 20 genres les plus fréquents
top_20_genres = genre_counts.nlargest(20).index

# Créer un nouveau DataFrame avec seulement les 20 genres les plus fréquents
top_20_genres_df = filtered_df[filtered_df['GENRE'].isin(top_20_genres)]

# Créer une table de contingence
contingency_table = pd.crosstab(top_20_genres_df['ARRONDISSEMENT'], top_20_genres_df['GENRE'])

# Créer la carte de chaleur
plt.figure(figsize=(10, 8))
sns.heatmap(contingency_table, cmap='plasma', annot=False, cbar=True)

# Ajouter le titre et les labels
plt.title("Répartition des genres d'arbres par arrondissement", fontsize=16)
plt.xlabel("GENRE", fontsize=14)
plt.ylabel("ARRONDISSEMENT", fontsize=14)

# Afficher le graphique
plt.show()

In [ ]:
# Vérifier et filtrer les valeurs NaN dans 'geo_point_2d'
filtered_df = filtered_df.dropna(subset=['geo_point_2d'])

# Extraire les coordonnées GPS et convertir en float
filtered_df[['lat', 'lon']] = filtered_df['geo_point_2d'].str.split(', ', expand=True).astype(float)

# Filtrer les lignes avec des valeurs NaN après la séparation des coordonnées
filtered_df = filtered_df.dropna(subset=['lat', 'lon'])

# Vérifiez le nom de la colonne contenant les données de hauteur
hauteur_colonne = 'HAUTEUR (m)'  # Changez cela si le nom de la colonne est différent

# Créer des points géométriques
filtered_df['geometry'] = filtered_df.apply(lambda row: Point(row['lon'], row['lat']), axis=1)

# Convertir le DataFrame en GeoDataFrame
gdf = gpd.GeoDataFrame(filtered_df, geometry='geometry')

# Définir le système de coordonnées en EPSG:4326 (WGS 84)
gdf.set_crs(epsg=4326, inplace=True)

# Reprojeter en EPSG:3857 pour l'affichage avec contextily
gdf = gdf.to_crs(epsg=3857)

# Créer la figure et l'axe
fig, ax = plt.subplots(figsize=(12, 10))

# Ajouter les points colorés en fonction de la hauteur
sc = gdf.plot(column=hauteur_colonne, cmap='plasma', markersize=5, ax=ax, legend=False)

# Ajouter le fond de carte de Paris avec contextily
ctx.add_basemap(ax, crs=gdf.crs.to_string(), source=ctx.providers.OpenStreetMap.Mapnik)

# Ajouter le titre
plt.title('Localisation des arbres selon leurs hauteur')

# Ajouter une colorbar verticale personnalisée
sm = plt.cm.ScalarMappable(cmap='plasma', norm=plt.Normalize(vmin=gdf[hauteur_colonne].min(), vmax=gdf[hauteur_colonne].max()))
sm._A = []
cbar = plt.colorbar(sm, ax=ax, orientation='vertical', pad=0.01, aspect=50)
cbar.set_label('Hauteur (m)', labelpad=-40, y=1.03, rotation=0)

# Enregistrer l'image
plt.savefig('paris_trees_height_map.png', dpi=300, bbox_inches='tight')

# Afficher le graphique
plt.show()

### 🔎 **VALEUR EMPIRIQUE**

In [11]:
# Fonction pour calculer les comptes et les pourcentages
def value_counts_with_percentage(series):
    counts = series.value_counts()
    percentages = series.value_counts(normalize=True) * 100
    return pd.DataFrame({'Counts': counts, 'Percentage': percentages})

# Compter les occurrences de chaque valeur unique dans la colonne 'typeemplacement'
TYPE_EMPLACEMENT = value_counts_with_percentage(df_no_duplicates['TYPE EMPLACEMENT'])
DOMANIALITE = value_counts_with_percentage(df_no_duplicates['DOMANIALITE'])
ARRONDISSEMENT = value_counts_with_percentage(df_no_duplicates['ARRONDISSEMENT'])
LIBELLE_FRANCAIS = value_counts_with_percentage(df_no_duplicates['LIBELLE FRANCAIS'])
GENRE = value_counts_with_percentage(df_no_duplicates['GENRE'])
ESPECE = value_counts_with_percentage(df_no_duplicates['ESPECE'])
VARIETE_OUCULTIVAR = value_counts_with_percentage(df_no_duplicates['VARIETE OUCULTIVAR'])
STADE_DE_DEVELOPPEMENT = value_counts_with_percentage(df_no_duplicates['STADE DE DEVELOPPEMENT'])

# Afficher les résultats
print(TYPE_EMPLACEMENT)
print("------------")
print(DOMANIALITE)
print("------------")
print(ARRONDISSEMENT)
print("------------")
print(LIBELLE_FRANCAIS)
print("------------")
print(GENRE)
print("------------")
print(ESPECE)
print("------------")
print(VARIETE_OUCULTIVAR)
print("------------")
print(STADE_DE_DEVELOPPEMENT)

                  Counts  Percentage
TYPE EMPLACEMENT                    
Arbre             210292       100.0
------------
              Counts  Percentage
DOMANIALITE                     
Alignement    108365   51.530729
Jardin         50845   24.178285
CIMETIERE      31868   15.154167
DASCO           7762    3.691058
PERIPHERIQUE    5122    2.435661
DJS             4781    2.273505
DFPE            1464    0.696175
DAC               67    0.031860
DASES             18    0.008560
------------
                   Counts  Percentage
ARRONDISSEMENT                       
PARIS 15E ARRDT     17756    8.443498
PARIS 13E ARRDT     17409    8.278489
PARIS 16E ARRDT     17112    8.137257
PARIS 20E ARRDT     15790    7.508607
PARIS 19E ARRDT     15157    7.207597
PARIS 12E ARRDT     13036    6.198999
SEINE-SAINT-DENIS   12117    5.761988
PARIS 17E ARRDT     11913    5.664980
BOIS DE VINCENNES   11817    5.619329
PARIS 14E ARRDT     11742    5.583665
PARIS 18E ARRDT     11200    5.325928
PARIS 

In [ ]:
# Compter les occurrences de chaque genre
genre_counts = filtered_df['ARRONDISSEMENT'].value_counts()

# Créer un nouveau Series pour le graphique
data = pd.concat([genre_counts])

# Calculer les pourcentages
data_percentage = data / data.sum() * 100

# Créer le graphique camembert
fig, ax = plt.subplots(figsize=(20, 18))
colors = plt.get_cmap('tab20').colors  # Utiliser un colormap avec 20 couleurs

# Définir les étiquettes pour être à l'intérieur
def func(pct, allvals):
    return f"{pct:.1f}%"

wedges, texts, autotexts = ax.pie(data_percentage, labels=None, autopct=lambda pct: func(pct, data_percentage),
                                  colors=colors, startangle=140, textprops=dict(color="w"))

# Ajouter les labels
for i, a in enumerate(autotexts):
    a.set_text(f"{data_percentage.index[i]}\n{a.get_text()}")

# Ajuster la taille du texte et éviter le chevauchement
for i, autotext in enumerate(autotexts):
    autotext.set_color("black")
    if data_percentage.iloc[i] < 1.2:
        autotext.set_fontsize(5)  # Taille du texte pour les segments < 4%
    elif data_percentage.iloc[i] < 4:
        autotext.set_fontsize(8)  # Taille du texte pour les segments < 4%            
    else:
        autotext.set_fontsize(14)  # Taille du texte pour les segments >= 4%

# Identifier les segments ayant des valeurs comprises entre 7.1% et 9.4%
for i, pct in enumerate(data_percentage):
    if 7 <= pct <= 10:
        autotexts[i].set_rotation_mode('anchor')
        angle = (wedges[i].theta2 - wedges[i].theta1) / 2 + wedges[i].theta1
        autotexts[i].set_rotation(angle + 180)
    elif pct < 7:
        autotexts[i].set_rotation_mode('anchor')
        angle = (wedges[i].theta2 - wedges[i].theta1) / 2 + wedges[i].theta1
        if angle > 180:
            angle -= 360
        autotexts[i].set_rotation(angle)
        # Ajuster la position du texte à la périphérie du camembert  
        if pct > 4:
            autotexts[i].set_position((0.72 * np.cos(np.deg2rad(angle)), 0.72 * np.sin(np.deg2rad(angle))))
        else:
            autotexts[i].set_position((0.85 * np.cos(np.deg2rad(angle)), 0.85 * np.sin(np.deg2rad(angle))))      

# Ajouter le titre
plt.title("Densité d'arbre par arrondissement")

# Ajouter une légende avec les pourcentages
legend_labels = [f'{name} ({pct:.1f}%)' for name, pct in zip(data.index, data_percentage)]
legend = ax.legend(wedges, legend_labels, title="Arrondissement", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))

# Afficher le graphique
plt.show()

In [ ]:
# Compter les occurrences de chaque genre
genre_counts = filtered_df['GENRE'].value_counts()

# Séparer les 20 premiers genres et regrouper les autres dans une catégorie "Other"
top_20_genres = genre_counts[:20]
other_genres = genre_counts[20:].sum()

# Créer un nouveau Series pour le graphique
data = pd.concat([top_20_genres, pd.Series({'Other': other_genres})])

# Calculer les pourcentages
data_percentage = data / data.sum() * 100

# Créer le graphique camembert
fig, ax = plt.subplots(figsize=(15, 12))
colors = plt.get_cmap('tab20').colors  # Utiliser un colormap avec 20 couleurs

# Définir les étiquettes pour être à l'intérieur
def func(pct, allvals):
    return f"{pct:.1f}%"

wedges, texts, autotexts = ax.pie(data_percentage, labels=None, autopct=lambda pct: func(pct, data_percentage),
                                  colors=colors, startangle=140, textprops=dict(color="w"))

# Ajouter les labels
for i, a in enumerate(autotexts):
    a.set_text(f"{data_percentage.index[i]}\n{a.get_text()}")

# Ajuster la taille du texte et éviter le chevauchement
for i, autotext in enumerate(autotexts):
    autotext.set_color("black")
    if data_percentage.iloc[i] < 1.2:
        autotext.set_fontsize(5)  # Taille du texte pour les segments < 4%
    elif data_percentage.iloc[i] < 4:
        autotext.set_fontsize(8)  # Taille du texte pour les segments < 4%            
    else:
        autotext.set_fontsize(14)  # Taille du texte pour les segments >= 4%

# Identifier les segments ayant des valeurs inférieures à 4%
for i, pct in enumerate(data_percentage):
    if pct < 4:
        autotexts[i].set_rotation_mode('anchor')
        angle = (wedges[i].theta2 - wedges[i].theta1) / 2 + wedges[i].theta1
        if angle > 180:
            angle -= 360
        autotexts[i].set_rotation(angle)
        # Ajuster la position du texte à la périphérie du camembert
        autotexts[i].set_position((0.93 * np.cos(np.deg2rad(angle)), 0.93 * np.sin(np.deg2rad(angle))))

# Ajouter le titre
plt.title("Types d'arbres")

# Ajouter une légende avec les pourcentages
legend_labels = [f'{name} ({pct:.1f}%)' for name, pct in zip(data.index, data_percentage)]
legend = ax.legend(wedges, legend_labels, title="Genres", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))

# Afficher le graphique
plt.show()

### 🔎 **REPONSE PROBLÉMATIQUE**

### I - NETTOYAGE DATA
Dans un premier temps on a pu constater que de nombreuse donnée était soit manquante ou très certainement fausse. Il serait donc nécessaire de vérifier ces dernières pour s'assurer de tirer des conclusions précises.

### II - OPTIMISATION APPROVISIONNEMENT & STOCKAGE
Nons avons pu analyser la quantité d'arbre par espèce, la répartition des densités ainsi que la dimension des arbres sur la région parisienne.
Avec ces différentes informations nous pouvont répartir plus efficacement le matériaux nécessaires pour l'entretien des arbres selon leur espèce ou leurs taille de façon géographique.

### III - INTENSIFICATION DE LA PLANTATION
Comme nous pouvons le voir sur le camenbert qui regroupe le nombre d'arbre par arrondissement les zones du 2ème, 3ème, 9ème arrondissement et des hauts de seine sont les moins denses et nécessitant en première une priorisation des plantations dans le futur.

### IV - DIVERSITÉ DES ARBRES
Nous pouvons constater sur le camenbert de type d'arbre qu'il y a 5 espèces qui sont prédominantes, représent plus de 63% des arbres plantés. On peut aussi accentuer la domination du Platanus qui représente à lui seul 22.9%.
Il pourrais être donc intéressant pour la diversité des espèces dans paris de ne plus planter au moins les 3 espèces qui représente plus de 10% du total et de privilégier les autres afins de rééquilibré la diversité.


### V - ARBRES MALADES
Un indicateur sur les arbres malades pourrait être déterminer par une hauteur et/ou un circonférence anormale par rapport à leur stade de développement.